In [22]:
import os

from pdf2image import convert_from_path
import shutil
import re

import pandas as pd
# pip install pandas
# pip install pdf2image
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# pip install img2table[surya]
# pip uninstall opencv-contrib-python opencv-python
# pip install opencv-contrib-python

from img2table.ocr import SuryaOCR
from img2table.document import Image

from surya.recognition import RecognitionPredictor
from surya.detection import DetectionPredictor


import surya

from PIL import Image as PilImage
import torch

import cv2

import numpy as np

In [23]:
class PdfOCR(SuryaOCR):
    from img2table.document.base import Document
    import typing
    def content(self, document: Document) -> typing.List["surya.recognition.schema.OCRResult"]:
        # Get OCR of all images
        ocrs = self.rec_predictor(images=[PilImage.fromarray(img) for img in document.images],
                                    # langs=[self.langs],
                                    det_predictor=self.det_predictor)

        return ocrs


In [24]:
pics_folder = 'pics'

In [25]:
pdf_folder = 'pdf'

pdf_files = os.listdir(pdf_folder)
pdf_files

['ON_SCHET_2BM-7704010978-770401001-201407030455299009568_2BM-7736520080-2012112806114018023400000000000_20250228_6ad2050b-3ccc-4db9-beff-fc3c7d766348.PDF',
 'æτÑΓ [F6PIF7].pdf']

In [26]:
# pdf_file_index = 0
pdf_file_index = 1

In [27]:
pdf_filepath = os.path.join(pdf_folder, pdf_files[pdf_file_index])
pdf_filepath

'pdf\\æτÑΓ [F6PIF7].pdf'

In [28]:
def images_from_pdf(pdf_filename, pics_folder):

    shutil.rmtree(pics_folder)
    os.mkdir(pics_folder)

    pages = convert_from_path(pdf_filename, 300)

    pics_files = []

    for i, page in enumerate(pages):
        pic_fname = 'page_{}.png'.format(i)
        pic_path = os.path.join(pics_folder, pic_fname)

        pics_files.append(pic_fname)
        # Save pages as images in the pdf
        page.save(pic_path, 'PNG')

        # enhance_image(pic_path)

    return pics_files

In [29]:
def is_numeric(x):

    result = 0
    
    if x is  None:
        result = 0
    elif isinstance(x, (int, float)):
        result = 1
    elif isinstance(x, str):
        x = x.replace(' ', '')
        x = x.replace('.', '')
        x = x.replace(',', '')

        result = 1 if x.isdigit() else 0

    return result

In [30]:
def get_number(x):
    result = None
    if isinstance(x, (int, float)):
        result = x
    elif isinstance(x, str):
        x = x.replace(' ', '')
        x = x.replace(',', '.')

        if x.replace('.', '').isdigit():
            if '.' in x:
                if x[0] == '.':
                    x = x[1:]

                result = float(x)
            else:
                result = int(x)

    return result 

In [31]:
c_df = None

In [32]:
def get_dataframes_from_pic(pic_filename, ocr, pics_folder):

    pic_filepath = os.path.join(pics_folder, pic_filename)
    print('-'*30)
    print(pic_filename)
    doc = Image(pic_filepath)

    extracted_tables = doc.extract_tables(ocr=ocr, min_confidence=10)

    result = []

    for ind, table in enumerate(extracted_tables):
        print('DF-{}'.format(ind))
        df = table.df
    #     if not check_df(df):
    #         df = None
    #     else:
    #         df = process_df(df)

        if df is not None:
            result.append(df)


    return result

In [33]:
if not os.path.exists('pics'):
    os.mkdir('pics')

In [34]:
def get_tables_and_text_from_pdf(pdf_filename, pics_folder='pics'):
    # Get images from pdf
    pic_files = images_from_pdf(pdf_filename, pics_folder)
    ocr = PdfOCR(langs=["ru"])
    recognition_predictor = RecognitionPredictor()
    detection_predictor = DetectionPredictor()

    dfs = []
    texts = []

    # ind = 5
    # pic_files = pic_files[ind:ind+1]
    
    for pic_file in pic_files:
        c_dfs = get_dataframes_from_pic(pic_file, ocr, pics_folder)

        dfs.extend(c_dfs)
        # pic_filepath = os.path.join(pics_folder, pic_file)
        # image = PilImage.open(pic_filepath)

        # predictions = recognition_predictor([image], det_predictor=detection_predictor)
        # texts.extend(predictions)

    

    return dfs, texts

In [35]:
dfs, texts = get_tables_and_text_from_pdf(pdf_filepath)

KeyboardInterrupt: 

In [ ]:
len(dfs)

2

In [ ]:
df = dfs[-1]
df

,0,1,2,3,4,5,6,7,8
0,"<math>\overline{\text{IITII}}</math>,",Тип расчета,Наименование\nуслуги,Адрес,ГП,Ед.<br>изм.,Коли-\nчество,Тариф,Сумма
1,01-07-158,Среднемесячный\nобъем потребления в Отопление\...,None,"Тепловая энергия Большая Переяславская\nул., д.7",ЖИЛЬЕ,Гкал,None,"61,404 3 217,19","164 623,61"
2,01-07-158,По расчету,Потери ТЭ,"Тепловая энергия Большая Переяславская\nул., д.7",ЖИЛЬЕ,Гкал,None,"1,610 3 217,19","4 3 1 6,40"
3,01-07-158,ПУ на доме,Вентиляция,"Тепловая энергия Большая Переяславская\nул., д.7",ПРОЧИЕ,Гкал,None,"51,597 2 680,99","138 331,04"
4,01-07-158,Среднемесячный\nобъем потребления\nв соответст...,Отопление\n95,"Тепловая энергия Большая Переяславская\nул., д.7",ПРОЧИЕ,Гкал,None,"22,182 2680,99","59 469,72"
5,01-07-158,По расчету,Потери ТЭ,"Тепловая энергия Большая Переяславская\nул., д.7",ПРОЧИЕ,Гкал,None,"1,420 | 2 680,99","3 807,01"
6,<math>01 - 09 -</math>\n0617/023,Среднемесячный\nобъем потребления в Отопление\...,None,"Тепловая энергия Большая Спасская ул.,\nд.12",ЖИЛЬЕ,Гкал,None,"140,836 3 217,19","377 580,13"
7,<math>01 - 09 -</math>\n0617/023,По расчету,Потери ТЭ,"Тепловая энергия Большая Спасская ул.,\nд.12",ЖИЛЬЕ,Гкал,None,"1,170 3 217,19","3 136,76"
8,<math>01 - 09 -</math>\n0617/023,ПУ на доме,Вентиляция,"Гепловая энергия Большая Спасская ул.,\nд.12",ПРОЧИЕ,Гкал,None,"15,049 2 680,99","40 346,22"
9,<math>01 - 09 -</math>\n0617/023,Среднемесячный\nобъем потребления\nв соответст...,Отопление,"Гепловая энергия Большая Спасская ул.,\nд.12",ПРОЧИЕ,Гкал,"10,065","2 680,99","26 984,16"


In [ ]:
df_2 = df.copy()

In [ ]:
def cleanup_column(cell):

    if cell is None:
        return ''
    
    result = cell.replace('\n', ' ')
    result = result.replace('- ', '')
    result = result.replace('<math>', '')
    result = result.replace('</math>', '')
    result = result.replace('<sup>', '')
    result = result.replace('</sup>', '')
    result = result.replace('<sub>', '')
    result = result.replace('</sub>', '')    
    result = result.replace('°', '')
    result = result.replace('<math display="inline">\\mathbf', '')    
      
    result = result.replace('|', '')
    match = re.search(r'\\mathbf{\d}', result)
    if match:
        result = result.replace(result[match.span()[1]-1], '')
        result = result.replace(r'\mathbf{', '')

    
    return result

In [ ]:
def process_number(cell):

    if cell is None:
        return 0
    
    if not cell:
        return 0
    
    result = cell.replace(',', '.').strip()
    result = result.replace(' ', '')    

    if '.' in result:
        result  = float(result)
    else:
        result = int(result)

    return result


In [ ]:
def process_table(ds, is_invoice = False):

    columns = list(ds.columns)
    
    # Очищаем от посторонних символов
    for col in columns:
        ds[col] = ds[col].apply(cleanup_column)

    if is_invoice:
        # Очищаем значения в соединенных колонках в шапке
        cc = [2, 5, 6, 7, 8, 9]
        ds.iloc[0, cc] = ''

        cc = [0, 3, 4, 10, 11]
        ds.iloc[1, cc] = ''

        cc = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        ds.iloc[3, cc] = ''        
        
        cc = [1, 2, 3, 4, 5, 6, 7, 8, 9]
        ds.iloc[3, cc] = '' 

        cc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
        ds.iloc[4, cc] = '' 

        # Обрабатываем числа
        cc = [10, 11]
        for c in cc:
            ds.iloc[3, c] = process_number(ds.iloc[3, c])

        cc = range(3, 12)
        for c in cc:
            ds.iloc[5, c] = process_number(ds.iloc[5, c])         

    return ds

In [ ]:
df_2 = process_table(df_2, is_invoice=pdf_file_index==0)
df_2

,0,1,2,3,4,5,6,7,8
0,"\overline{\text{IITII}},",Тип расчета,Наименование услуги,Адрес,ГП,Ед.<br>изм.,Количество,Тариф,Сумма
1,01-07-158,Среднемесячный объем потребления в Отопление с...,,"Тепловая энергия Большая Переяславская ул., д.7",ЖИЛЬЕ,Гкал,,"61,404 3 217,19","164 623,61"
2,01-07-158,По расчету,Потери ТЭ,"Тепловая энергия Большая Переяславская ул., д.7",ЖИЛЬЕ,Гкал,,"1,610 3 217,19","4 3 1 6,40"
3,01-07-158,ПУ на доме,Вентиляция,"Тепловая энергия Большая Переяславская ул., д.7",ПРОЧИЕ,Гкал,,"51,597 2 680,99","138 331,04"
4,01-07-158,Среднемесячный объем потребления в соответстви...,Отопление 95,"Тепловая энергия Большая Переяславская ул., д.7",ПРОЧИЕ,Гкал,,"22,182 2680,99","59 469,72"
5,01-07-158,По расчету,Потери ТЭ,"Тепловая энергия Большая Переяславская ул., д.7",ПРОЧИЕ,Гкал,,"1,420 2 680,99","3 807,01"
6,01 09 - 0617/023,Среднемесячный объем потребления в Отопление с...,,"Тепловая энергия Большая Спасская ул., д.12",ЖИЛЬЕ,Гкал,,"140,836 3 217,19","377 580,13"
7,01 09 - 0617/023,По расчету,Потери ТЭ,"Тепловая энергия Большая Спасская ул., д.12",ЖИЛЬЕ,Гкал,,"1,170 3 217,19","3 136,76"
8,01 09 - 0617/023,ПУ на доме,Вентиляция,"Гепловая энергия Большая Спасская ул., д.12",ПРОЧИЕ,Гкал,,"15,049 2 680,99","40 346,22"
9,01 09 - 0617/023,Среднемесячный объем потребления в соответстви...,Отопление,"Гепловая энергия Большая Спасская ул., д.12",ПРОЧИЕ,Гкал,"10,065","2 680,99","26 984,16"
